In [1]:
import numpy
from matplotlib import pyplot
from pandas import read_csv
import math
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
%matplotlib inline
dataframe1 = read_csv('newDailyData.csv', usecols = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],engine='python')
dataframe = read_csv('newDailyData.csv',engine='python')
dataframe = dataframe[['cmchigh','cmclow','cmcclose','cmcvolume']]
dataset1 = dataframe1.values
# 将整型变为float
dataset1 = dataset1.astype('float64')
dataset = dataframe.values
# 将整型变为float
dataset = dataset.astype('float64')

In [2]:
print(dataframe)

      cmchigh   cmclow  cmcclose    cmcvolume
0     2397.17  2260.23  17776.70  16894499840
1     2900.70  2269.89  16624.60  22149699584
2     2838.41  2621.85  15802.90  16516599808
3     2862.42  2657.71  13831.80  22197999616
4     2832.18  2653.94  14699.20  13086000128
..        ...      ...       ...          ...
829  10021.74  8662.62   6469.80  34585598367
830   9749.53  9112.54   6242.19  34885225901
831   9805.12  9256.15   5922.04  28373690931
832   9516.18  9232.65   6429.84  37101651525
833   9426.87  9085.37   6438.64  32786468812

[834 rows x 4 columns]


In [3]:
dataset.shape

(834, 4)

In [4]:
dataframe.isnull().sum()

cmchigh      0
cmclow       0
cmcclose     0
cmcvolume    0
dtype: int64

In [5]:
dataset

array([[2.39717000e+03, 2.26023000e+03, 1.77767000e+04, 1.68944998e+10],
       [2.90070000e+03, 2.26989000e+03, 1.66246000e+04, 2.21496996e+10],
       [2.83841000e+03, 2.62185000e+03, 1.58029000e+04, 1.65165998e+10],
       ...,
       [9.80512000e+03, 9.25615000e+03, 5.92204000e+03, 2.83736909e+10],
       [9.51618000e+03, 9.23265000e+03, 6.42984000e+03, 3.71016515e+10],
       [9.42687000e+03, 9.08537000e+03, 6.43864000e+03, 3.27864688e+10]])

In [6]:
import numpy as np
def ema(x, p):
    x = x.squeeze()
    prev_ema = x[:p].mean()
    ema = [prev_ema]
    m = len(x)
    multiplier = 2 / float(p + 1)
    for i in range(p, m):
        cur_ema = (x[i] - prev_ema) * multiplier + prev_ema
        prev_ema = cur_ema
        ema.append(cur_ema)
    return np.array(ema)
def getRSI(x):
    x = x.squeeze()
    n = len(x)
    x0 = x[:n - 1]
    x1 = x[1:]
    change = x1 - x0
    avgGain = []
    avgLoss = []
    loss = 0
    gain = 0
    for i in range(14):
        if change[i] > 0:
            gain += change[i]
        elif change[i] < 0:
            loss += abs(change[i])
    averageGain = gain / 14.0
    averageLoss = loss / 14.0
    avgGain.append(averageGain)
    avgLoss.append(averageLoss)
    for i in range(14, n - 1):
        if change[i] >= 0:
            avgGain.append((avgGain[-1] * 13 + change[i]) / 14.0)
            avgLoss.append((avgLoss[-1] * 13) / 14.0)
        else:
            avgGain.append((avgGain[-1] * 13) / 14.0)
            avgLoss.append((avgLoss[-1] * 13 + abs(change[i])) / 14.0)
    avgGain = np.array(avgGain)
    avgLoss = np.array(avgLoss)
    RS = avgGain / avgLoss
    RSI = 100 - (100 / (1 + RS))
    return np.c_[RSI, x1[13:]]


def getStochasticOscillator(x):
    high = x[:, 0].squeeze()
    low = x[:, 1].squeeze()
    close = x[:, 2].squeeze()
    n = len(high)
    highestHigh = []
    lowestLow = []
    for i in range(n - 13):
        highestHigh.append(high[i:i + 14].max())
        lowestLow.append(low[i:i + 14].min())
    highestHigh = np.array(highestHigh)
    lowestLow = np.array(lowestLow)
    k = 100 * ((close[13:] - lowestLow) / (highestHigh - lowestLow))

    return np.c_[k, close[13:]]


def getWilliams(x):
    high = x[:, 0].squeeze()
    low = x[:, 1].squeeze()
    close = x[:, 2].squeeze()
    n = len(high)
    highestHigh = []
    lowestLow = []
    for i in range(n - 13):
        highestHigh.append(high[i:i + 14].max())
        lowestLow.append(low[i:i + 14].min())
    highestHigh = np.array(highestHigh)
    lowestLow = np.array(lowestLow)
    w = -100 * ((highestHigh - close[13:]) / (highestHigh - lowestLow))
    return np.c_[w, close[13:]]


def getMACD(close):
    ma1 = ema(close.squeeze(), 12)
    ma2 = ema(close.squeeze(), 26)
    macd = ma1[14:] - ma2
    return np.c_[macd, close[len(close) - len(macd):]]


def getPriceRateOfChange(close, n_days):
    close = close.squeeze()
    n = len(close)
    x0 = close[:n - n_days]
    x1 = close[n_days:]
    PriceRateOfChange = (x1 - x0) / x0
    return np.c_[PriceRateOfChange, x1]


def getOnBalanceVolume(X):
    close = X[:, 2].squeeze()
    volume = X[:, 3].squeeze()[1:]
    n = len(close)
    x0 = close[:n - 1]
    x1 = close[1:]
    change = x1 - x0
    OBV = []
    prev_OBV = 0

    for i in range(n - 1):
        if change[i] > 0:
            current_OBV = prev_OBV + volume[i]
        elif change[i] < 0:
            current_OBV = prev_OBV - volume[i]
        else:
            current_OBV = prev_OBV
        OBV.append(current_OBV)
        prev_OBV = current_OBV
    OBV = np.array(OBV)
    return np.c_[OBV, x1]

def getTechnicalIndicators(X, d):
    RSI = getRSI(X[:, 2])
    StochasticOscillator = getStochasticOscillator(X)
    Williams = getWilliams(X)

    MACD = getMACD(X[:, 2])
    PROC = getPriceRateOfChange(X[:, 2], d)
    OBV = getOnBalanceVolume(X)

    min_len = min(len(RSI),
                  len(StochasticOscillator),
                  len(Williams),
                  len(MACD),
                  len(PROC),
                  len(OBV))

    RSI = RSI[len(RSI) - min_len:]
    StochasticOscillator = StochasticOscillator[len(StochasticOscillator) - min_len:]
    Williams = Williams[len(Williams) - min_len:]
    MACD = MACD[len(MACD) - min_len:]
    PROC = PROC[len(PROC) - min_len:]
    OBV = OBV[len(OBV) - min_len:]

    feature_matrix = np.c_[RSI[:, 0],
                           StochasticOscillator[:, 0],
                           Williams[:, 0],
                           MACD[:, 0],
                           PROC[:, 0],
                           OBV[:, 0]]

    return feature_matrix

In [8]:
feature_matrix = getTechnicalIndicators(dataset,2)
feature_matrix.shape

(809, 6)

In [9]:
newdataset = numpy.hstack((feature_matrix,dataset1[-809:,:]))

In [10]:
y1 = dataframe.iloc[-809:,2].values
y2=dataframe.iloc[-809-1:-1,2].values
labels = numpy.sign(y1-y2)
labels

array([ 1., -1.,  1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,
       -1.,  1.,  1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,
        1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
       -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
       -1., -1., -1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,
        1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1.,
       -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,
        1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,
       -1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,
       -1., -1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1.,
       -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
        1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,
        1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,
       -1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1., -1

In [11]:
labels.shape

(809,)

In [12]:
from collections import  Counter
Counter(labels)

Counter({1.0: 415, -1.0: 394})

In [13]:
def create_dataset(dataset, dataset1,look_back,look_lag,temp):
    dataX, dataY = [], []
    for i in range(0,len(dataset)-look_back-look_lag+1,temp):
        b = dataset1[(i+look_back):(i+look_back+look_lag)]
        if(b==0):
            continue
        else:
            dataY.append(b)
            a = dataset[i:(i+look_back), :]
            dataX.append(a)
        
       
    return numpy.array(dataX), numpy.array(dataY)

In [14]:
from sklearn.model_selection import train_test_split
look_back = 3
look_lag = 1 #用前20个数据预测当前1个数据
train_size =606
test_size = len(feature_matrix) - train_size
train, test = newdataset[0:train_size,:], newdataset[train_size-look_back:len(feature_matrix),:]
train1, test1 = labels[0:train_size], labels[train_size-look_back:len(feature_matrix)]
trainX, trainY = create_dataset(train, train1,look_back,look_lag,1)
testX, testY = create_dataset(test, test1,look_back,look_lag,1)
print(trainX.shape)
features_set = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1]*29))
scaler = StandardScaler()
features_set_scaler = scaler.fit_transform(features_set)
test_features = numpy.reshape(testX, (testX.shape[0], testX.shape[1]*29))
test_features_scaler = scaler.fit_transform(test_features)

trainY = trainY.reshape(trainY.shape[0])
testY = testY.reshape(testY.shape[0])


def result(testY,testPredict):
    from sklearn.metrics import accuracy_score,classification_report
    acc = accuracy_score(testY,testPredict)
    report = classification_report(testY,testPredict)
    print("acc:",acc)
    print(report)

(603, 3, 29)


In [15]:
from sklearn.linear_model import LogisticRegression as LR
model1 = LR(max_iter=1000) 
'''
C=100,random_state=0, solver='lbfgs', max_iter=1000,#lbfgs
                         multi_class='multinomial'
'''
model1.fit(features_set_scaler, trainY)
testPredict1 = model1.predict(test_features_scaler)  

In [16]:
result(testY,testPredict1)

acc: 0.49261083743842365
              precision    recall  f1-score   support

        -1.0       0.55      0.41      0.47       111
         1.0       0.45      0.60      0.52        92

    accuracy                           0.49       203
   macro avg       0.50      0.50      0.49       203
weighted avg       0.51      0.49      0.49       203



In [17]:
from sklearn.linear_model import LogisticRegression as LR
model1 = LR(max_iter=1000) 
'''
C=100,random_state=0, solver='lbfgs', max_iter=1000,#lbfgs
                         multi_class='multinomial'
'''
model1.fit(features_set, trainY)
testPredict1 = model1.predict(test_features)  
result(testY,testPredict1)

acc: 0.49261083743842365
              precision    recall  f1-score   support

        -1.0       0.72      0.12      0.20       111
         1.0       0.47      0.95      0.63        92

    accuracy                           0.49       203
   macro avg       0.60      0.53      0.41       203
weighted avg       0.61      0.49      0.39       203



In [18]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model2 = LinearDiscriminantAnalysis() 
'''
solver='lsqr', shrinkage=None, priors=None, n_components=None, store_covariance=False
'''
model2.fit(features_set_scaler, trainY)
testPredict2 = model2.predict(test_features_scaler)  
result(testY,testPredict2)

acc: 0.4876847290640394
              precision    recall  f1-score   support

        -1.0       0.54      0.42      0.47       111
         1.0       0.45      0.57      0.50        92

    accuracy                           0.49       203
   macro avg       0.49      0.49      0.49       203
weighted avg       0.50      0.49      0.49       203



In [19]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model2 = LinearDiscriminantAnalysis() 
'''
solver='lsqr', shrinkage=None, priors=None, n_components=None, store_covariance=False
'''
model2.fit(features_set, trainY)
testPredict2 = model2.predict(test_features)  
result(testY,testPredict2)

acc: 0.5270935960591133
              precision    recall  f1-score   support

        -1.0       0.54      0.94      0.68       111
         1.0       0.30      0.03      0.06        92

    accuracy                           0.53       203
   macro avg       0.42      0.48      0.37       203
weighted avg       0.43      0.53      0.40       203



In [20]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model3 = QuadraticDiscriminantAnalysis() 
model3.fit(features_set, trainY)
testPredict3 = model3.predict(test_features)  
result(testY,testPredict3)

acc: 0.5221674876847291
              precision    recall  f1-score   support

        -1.0       0.55      0.68      0.61       111
         1.0       0.46      0.33      0.38        92

    accuracy                           0.52       203
   macro avg       0.51      0.51      0.50       203
weighted avg       0.51      0.52      0.51       203



In [21]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model3 = QuadraticDiscriminantAnalysis() 
model3.fit(features_set_scaler, trainY)
testPredict3 = model3.predict(test_features_scaler)  
result(testY,testPredict3)

acc: 0.5467980295566502
              precision    recall  f1-score   support

        -1.0       0.55      1.00      0.71       111
         1.0       0.00      0.00      0.00        92

    accuracy                           0.55       203
   macro avg       0.27      0.50      0.35       203
weighted avg       0.30      0.55      0.39       203



/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
!pip install xgboost

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [22]:
import xgboost as xgb
model4 = xgb.XGBClassifier()
'''
objective ='reg:logistic', colsample_bytree = 0.5, learning_rate = 0.003,
                max_depth = 20, alpha = 20, n_estimators = 400
'''
model4.fit(features_set_scaler, trainY)
testPredict4 = model4.predict(test_features_scaler)  
result(testY,testPredict4)

acc: 0.5073891625615764
              precision    recall  f1-score   support

        -1.0       0.56      0.48      0.51       111
         1.0       0.46      0.54      0.50        92

    accuracy                           0.51       203
   macro avg       0.51      0.51      0.51       203
weighted avg       0.51      0.51      0.51       203



In [23]:
import xgboost as xgb
model4 = xgb.XGBClassifier()
'''
objective ='reg:logistic', colsample_bytree = 0.5, learning_rate = 0.003,
                max_depth = 20, alpha = 20, n_estimators = 400
'''
model4.fit(features_set, trainY)
testPredict4 = model4.predict(test_features)  
result(testY,testPredict4)

acc: 0.5369458128078818
              precision    recall  f1-score   support

        -1.0       0.56      0.74      0.64       111
         1.0       0.48      0.29      0.36        92

    accuracy                           0.54       203
   macro avg       0.52      0.52      0.50       203
weighted avg       0.52      0.54      0.51       203



In [24]:
from sklearn import svm
model5 = svm.SVC()
model5.fit(features_set_scaler, trainY)
testPredict5 = model5.predict(test_features_scaler)  
result(testY,testPredict5)

acc: 0.5270935960591133
              precision    recall  f1-score   support

        -1.0       0.59      0.43      0.50       111
         1.0       0.48      0.64      0.55        92

    accuracy                           0.53       203
   macro avg       0.54      0.54      0.53       203
weighted avg       0.54      0.53      0.52       203



In [25]:
from sklearn import svm
model5 = svm.SVC()
model5.fit(features_set, trainY)
testPredict5 = model5.predict(test_features)  
result(testY,testPredict5)

acc: 0.5467980295566502
              precision    recall  f1-score   support

        -1.0       0.55      1.00      0.71       111
         1.0       0.00      0.00      0.00        92

    accuracy                           0.55       203
   macro avg       0.27      0.50      0.35       203
weighted avg       0.30      0.55      0.39       203



/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier()
regr.fit(features_set_scaler, trainY)

RandomForestClassifier()

In [27]:
testPredict6 = regr.predict(test_features_scaler)  
result(testY,testPredict6)

acc: 0.5123152709359606
              precision    recall  f1-score   support

        -1.0       0.57      0.45      0.50       111
         1.0       0.47      0.59      0.52        92

    accuracy                           0.51       203
   macro avg       0.52      0.52      0.51       203
weighted avg       0.52      0.51      0.51       203



In [28]:
from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier()
regr.fit(features_set, trainY)
testPredict6 = regr.predict(test_features)  
result(testY,testPredict6)

acc: 0.4975369458128079
              precision    recall  f1-score   support

        -1.0       0.54      0.58      0.56       111
         1.0       0.44      0.40      0.42        92

    accuracy                           0.50       203
   macro avg       0.49      0.49      0.49       203
weighted avg       0.49      0.50      0.49       203



In [29]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [30]:
trainY1 = trainY.copy()
trainY1[trainY1==-1]=0

testY1 = testY.copy()
testY1[testY1==-1]=0

In [39]:
Scaler2 = MinMaxScaler(feature_range=(0, 1))
feature_matrix1=Scaler2.fit_transform(feature_matrix)
look_back = 3
look_lag = 1 #用前20个数据预测当前1个数据
train_size = 606
test_size = len(feature_matrix1) - train_size
train, test = feature_matrix1[0:train_size,:], feature_matrix1[train_size-look_back:len(feature_matrix1),:]
train1, test1 = labels[0:train_size], labels[train_size-look_back:len(feature_matrix1)]
trainX1, trainY2 = create_dataset(train, train1,look_back,look_lag,1)
testX1, testY2 = create_dataset(test, test1,look_back,look_lag,1)

In [40]:
trainX

array([[[4.17900449e+01, 7.59294064e+02, 6.59294064e+02, ...,
         2.57226304e+09, 1.36509770e+06, 6.52050000e+05],
        [3.97208517e+01, 6.70277204e+02, 5.70277204e+02, ...,
         2.64824758e+09, 1.26917280e+06, 5.92042000e+05],
        [3.99809305e+01, 6.23885799e+02, 5.23885799e+02, ...,
         2.79149312e+09, 1.49781017e+06, 6.70840000e+05]],

       [[3.97208517e+01, 6.70277204e+02, 5.70277204e+02, ...,
         2.64824758e+09, 1.26917280e+06, 5.92042000e+05],
        [3.99809305e+01, 6.23885799e+02, 5.23885799e+02, ...,
         2.79149312e+09, 1.49781017e+06, 6.70840000e+05],
        [3.25996313e+01, 4.97244795e+02, 3.97244795e+02, ...,
         3.28004038e+09, 2.88928659e+06, 7.37399000e+05]],

       [[3.99809305e+01, 6.23885799e+02, 5.23885799e+02, ...,
         2.79149312e+09, 1.49781017e+06, 6.70840000e+05],
        [3.25996313e+01, 4.97244795e+02, 3.97244795e+02, ...,
         3.28004038e+09, 2.88928659e+06, 7.37399000e+05],
        [3.17806664e+01, 4.85064734e

In [41]:
trainX1

array([[[0.40392724, 0.29964799, 0.29964799, 0.49574312, 0.73146838,
         0.08196649],
        [0.37770168, 0.28219293, 0.28219293, 0.46845741, 0.58920067,
         0.0670282 ],
        [0.380998  , 0.27309617, 0.27309617, 0.44988582, 0.55167985,
         0.08421273]],

       [[0.37770168, 0.28219293, 0.28219293, 0.46845741, 0.58920067,
         0.0670282 ],
        [0.380998  , 0.27309617, 0.27309617, 0.44988582, 0.55167985,
         0.08421273],
        [0.28744526, 0.2482635 , 0.2482635 , 0.36455578, 0.34016197,
         0.05880306]],

       [[0.380998  , 0.27309617, 0.27309617, 0.44988582, 0.55167985,
         0.08421273],
        [0.28744526, 0.2482635 , 0.2482635 , 0.36455578, 0.34016197,
         0.05880306],
        [0.27706546, 0.24587515, 0.24587515, 0.29083543, 0.29930365,
         0.03342464]],

       ...,

       [[0.55898268, 0.24561388, 0.24561388, 0.55615354, 0.60638483,
         0.90921238],
        [0.51239504, 0.24244   , 0.24244   , 0.56139617, 0.57624013,
  

In [44]:
import numpy
import keras
from matplotlib import pyplot
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Conv1D,Flatten
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
from keras.models import load_model
%matplotlib inline
from keras import optimizers,initializers,regularizers
model = Sequential() 

model.add(LSTM(units=16, return_sequences=True, 
               input_shape=(trainX1.shape[1],trainX1.shape[2])))#,kernel_regularizer=regularizers.l2(0.01)
#model.add(Dropout(0.3))

model.add(LSTM(units=4, return_sequences=True,activation='relu'))#,kernel_regularizer=regularizers.l2(0.01)
#model.add(Dropout(0.2))

model.add(LSTM(units=4,return_sequences=False,activation='relu'))#,kernel_regularizer=regularizers.l2(0.01)
#model.add(Dropout(0.1))

model.add(Dense(units = 1,activation='sigmoid'))#sigmoid
model.compile(loss='binary_crossentropy',#binary_crossentropy
                  optimizer='Adam',#RMSprop optimizers.Adam(lr=0.00001)
                  metrics=['accuracy'])  #mean_squared_error metrics=['accuracy']
history = model.fit(trainX1,trainY1,epochs=500,batch_size=120, shuffle=True)

Epoch 1/500
603/603 [==============================] - 3s 5ms/step - loss: 0.6931 - acc: 0.5274
Epoch 2/500
603/603 [==============================] - 0s 193us/step - loss: 0.6929 - acc: 0.5323
Epoch 3/500
603/603 [==============================] - 0s 192us/step - loss: 0.6928 - acc: 0.5323
Epoch 4/500
603/603 [==============================] - 0s 193us/step - loss: 0.6926 - acc: 0.5323
Epoch 5/500
603/603 [==============================] - 0s 188us/step - loss: 0.6926 - acc: 0.5323
Epoch 6/500
603/603 [==============================] - 0s 195us/step - loss: 0.6925 - acc: 0.5323
Epoch 7/500
603/603 [==============================] - 0s 189us/step - loss: 0.6925 - acc: 0.5323
Epoch 8/500
603/603 [==============================] - 0s 190us/step - loss: 0.6924 - acc: 0.5323
Epoch 9/500
603/603 [==============================] - 0s 192us/step - loss: 0.6924 - acc: 0.5323
Epoch 10/500
603/603 [==============================] - 0s 190us/step - loss: 0.6923 - acc: 0.5323
Epoch 11/500
603/603 

In [45]:
testPredict = model.predict_classes(testX1)
result(testY1,testPredict)

acc: 0.45320197044334976
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       111
         1.0       0.45      1.00      0.62        92

    accuracy                           0.45       203
   macro avg       0.23      0.50      0.31       203
weighted avg       0.21      0.45      0.28       203



/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
